In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split

In [ ]:
csv_file_path = '/kaggle/input/flood-data/Kaggle/devset_images_gt.csv'
df = pd.read_csv(csv_file_path)


image_folder_path = '/kaggle/input/flood-data/Kaggle/devset_images/devset_images'
image_files = [file for file in os.listdir(image_folder_path) if not file.startswith('._') and file.endswith('.jpg')]


selected_image_files = []
selected_labels = []

for image_file in image_files:
    image_id = int(image_file.split('.')[0])  
    label = df[df['id'] == image_id]['label'].values
    if len(label) > 0:
        selected_image_files.append(image_file)
        selected_labels.append(label[0])


X = []
y = []

for image_file, label in zip(selected_image_files, selected_labels):
    image_path = os.path.join(image_folder_path, image_file)
    image = Image.open(image_path)
    image = image.resize((224, 224))  
    image = np.array(image)  
    X.append(image)
    y.append(label)

X = np.array(X)
y = np.array(y)


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [ ]:

def create_efficientnetb7_model_complex():
    base_model = tf.keras.applications.EfficientNetB7(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.25)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [ ]:
effnet_model = create_efficientnetb7_model_complex()

effnet_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
effnet_model.summary()

In [ ]:
epochs = 40
batch_size = 32

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:


from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

import math
def lr_schedule(epoch):
    initial_lr = 0.001  
    drop = 0.5  
    epochs_drop = 10  
    new_lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return new_lr
optimizer = RMSprop(learning_rate=0.001)
lr_scheduler = LearningRateScheduler(lr_schedule)


In [ ]:


effnet_history = effnet_model.fit(
    datagen.flow(X_train, y_train, batch_size=batch_size),  
    steps_per_epoch=len(X_train) // batch_size,
    epochs=epochs,
    validation_data=(X_test, y_test),  
    callbacks=[lr_scheduler] 
)


In [ ]:
loss_effnet, accuracy_effnet = effnet_model.evaluate(X_test, y_test)

print("EfficientNetB7 - Loss:", loss_effnet, "Accuracy:", accuracy_effnet)

In [ ]:
import matplotlib.pyplot as plt


plt.plot(effnet_history.history['accuracy'])
plt.plot(effnet_history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


plt.plot(effnet_history.history['loss'])
plt.plot(effnet_history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image


test_image_folder_path = '/kaggle/input/flood-data/Kaggle/testset_images/testset_images'
test_image_files = [file for file in os.listdir(test_image_folder_path) if not file.startswith('._')]
X_test = []

for image_file in test_image_files:
    image_id_str = image_file.split('.')[0]
    image_id = int(image_id_str)
    image_path = os.path.join(test_image_folder_path, image_file)
    image = Image.open(image_path)
    image = image.resize((224, 224))
    image = np.array(image)
    X_test.append(image)

X_test = np.array(X_test)

test_predictions = effnet_model.predict(X_test)
binary_predictions = (test_predictions > 0.5).astype(int).flatten()

test_df = pd.DataFrame({'id': [int(file.split('.')[0]) for file in test_image_files],
                        'label': binary_predictions})

test_predictions_csv_path = '/kaggle/working/test_predictions.csv'
test_df.to_csv(test_predictions_csv_path, index=False)

print(f'Test predictions saved to {test_predictions_csv_path}')
